# AES-128 Implementation in Python
The objective of this project was to implement the Advanced Encryption Standard (AES-128) in Python. The implementation is clarity-forward by representing data as both integer and string values throughout the code, allowing better understanding and visualization of the encryption and decryption processes. Core Operations which SubBytes, ShiftRows, MixColumns, AddRoundKey, Gallois Matrix Operations etc. have been written to accommodate the above data representation

In [1]:
import numpy as np
from aes_tables import *   

## Helper Functions: Int <-> String <-> Binary 

We currently work on 16 x 16 bit sentences. We've decided to convert things from string to ASCII Binary and then convert it to integers. This integers will lie between 0 & 255 of x mod(256)

In [2]:
def Cvt_Bin(char):
    ascii_val = ord(char)
    binary_val = format(ascii_val, '08b')
    return binary_val

def Cvt_Bin_from_int(integer):
    binary_str = format(int(integer), '08b')  # ensure int
    return binary_str

def Cvt_int_from_Bin(Binary: list):
    binary_str = "".join(Binary)
    integer = int(binary_str, 2)
    return integer

In [3]:
def Binary_List(string):
    # Pad to 16 characters (left padding as you had it)
    while len(string) < 16:
        string =string + " "

    int_list = []
    for char in string:
        # EDIT: Removed .upper() to preserve original bytes
        binary_str = Cvt_Bin(char)
        int_val = int(binary_str, 2)  # Convert binary to integer (0–255)
        int_list.append(int_val)

    return int_list

In [4]:
def Parcellate(X: str):
    i = 16
    Chunks = []
    while(i < len(X)):
        Chunks.append(X[i-16: i])
        i += 16
    if i - 16 < len(X):
        Chunks.append(X[i-16:])
    return Chunks

## Helper Functions: Rotate & Substitution 

In [5]:
def Rotate_Word(word: np.array) -> np.array:
    return np.roll(word, -1)

def SBox(key: np.array, encrypt=True) -> np.array:
    table = S_BOX if encrypt else INV_S_BOX
    return np.array([table[int(b)] for b in key], dtype=np.uint8)

def INV_Rotate_Word(word: np.array) -> np.array:
    return np.roll(word, +1)

In [6]:
def Get_RoundKey(Keys, start):
    # Keys[start:start+4] is list of 4 words (each a 4-byte np.array)
    # Build 4x4 column-major matrix for AddRoundKey
    return np.concatenate(Keys[start:start+4]).reshape(4, 4, order='F')

## Round Key Generator: 

In [7]:
def Generate_All_Keys(W):
    i = 4
    RCON = [1, 2, 4, 8, 16, 32, 64, 128, 27, 54]

    while len(W) < 44:
        temp = W[-1].copy()

        if i % 4 == 0:
            temp = Rotate_Word(temp)
            temp = SBox(temp, True)
            temp[0] ^= RCON[i//4 - 1]

        new_word = np.bitwise_xor(W[i - 4], temp)
        W.append(new_word)
        i += 1

    return W

## Mix Row Functions: 

In [8]:
def G(Input):
    Guy = [
        [2, 3, 1, 1],
        [1, 2, 3, 1],
        [1, 1, 2, 3],
        [3, 1, 1, 2]
    ]

    def xtime(integer):
        b = int(integer) & 0xFF
        carry = (b & 0x80) != 0
        b = (b << 1) & 0xFF
        if carry:
            b ^= 0x1B
        return b

    Final = []

    for row in range(4):
        List = []
        for g_column in range(4):
            S = 0
            for item in range(4):
                coef = Guy[row][item]
                val  = int(Input[item][g_column])
                if coef == 1:
                    S ^= val
                elif coef == 2:
                    S ^= xtime(val)
                elif coef == 3:
                    S ^= (xtime(val) ^ val)
            List.append(S & 0xFF)
        Final.append(List)

    return np.array(Final, dtype=np.uint8)

In [9]:
def INV_G(Input):
    Guy = [
        [14, 11, 13, 9],
        [9, 14, 11, 13],
        [13, 9, 14, 11],
        [11, 13, 9, 14]
    ]

    def xtime(b):
        b &= 0xFF
        carry = b & 0x80
        b = (b << 1) & 0xFF
        if carry:
            b ^= 0x1B
        return b

    def mul(val, coef):
        if coef == 9:
            x2 = xtime(val)
            x4 = xtime(x2)
            x8 = xtime(x4)
            return x8 ^ val
        elif coef == 11:
            x2 = xtime(val)
            x4 = xtime(x2)
            x8 = xtime(x4)
            return x8 ^ x2 ^ val
        elif coef == 13:
            x2 = xtime(val)
            x4 = xtime(x2)
            x8 = xtime(x4)
            return x8 ^ x4 ^ val
        elif coef == 14:
            x2 = xtime(val)
            x4 = xtime(x2)
            x8 = xtime(x4)
            return x8 ^ x4 ^ x2
        else:
            raise ValueError("Invalid coefficient in InvMixColumns")

    Final = []
    for row in range(4):
        L = []
        for col in range(4):
            S = 0
            for k in range(4):
                coef = Guy[row][k]
                val  = int(Input[k][col])
                S ^= mul(val, coef)
            L.append(S)
        Final.append(L)

    return np.array(Final, dtype=np.uint8)

# Encryption: 

In [10]:
def Encryption(X, Keys):
    start = 0
    end = 4

    RoundKey = Get_RoundKey(Keys, start)
    state = X ^ RoundKey

    start = end
    end += 4

    for i in range(1, 10):
        state = state.flatten(order='F')
        state = SBox(state, True).reshape(4, 4, order='F')

        # EDIT: Simplified ShiftRows
        for j in range(1, 4):
            state[j] = np.roll(state[j], -j)

        state = G(state)

        RoundKey = Get_RoundKey(Keys, start)
        state = state ^ RoundKey
        start = end
        end += 4

    # Final round (no MixColumns)
    state = state.flatten(order='F')
    state = SBox(state, True).reshape(4, 4, order='F')

    for j in range(1, 4):
        state[j] = np.roll(state[j], -j)  # EDIT: simplified

    RoundKey = Get_RoundKey(Keys, start)
    state = state ^ RoundKey

    return state.flatten(order='F')


# Decryption: 

In [11]:
def Decryption(X, Keys):
    start = 40
    end   = 44

    RoundKey = Get_RoundKey(Keys, start)
    state = X ^ RoundKey

    start -= 4
    end   -= 4

    for i in range(9, 0, -1):
        # EDIT: Simplified InvShiftRows
        for j in range(1, 4):
            state[j] = np.roll(state[j], j)

        state = state.flatten(order='F')
        state = SBox(state, False).reshape(4, 4, order='F')

        RoundKey = Get_RoundKey(Keys, start)
        state = state ^ RoundKey

        start -= 4
        end   -= 4

        state = INV_G(state)

    # Final round (NO InvMixColumns)
    for j in range(1, 4):
        state[j] = np.roll(state[j], j)

    state = state.flatten(order='F')
    state = SBox(state, False).reshape(4, 4, order='F')

    RoundKey = Get_RoundKey(Keys, start)
    state = state ^ RoundKey

    return state.flatten(order='F')

In [25]:
plaintext = "HELLO THIS IS A TEST OF AES LONG MESSAGE!"
Parcellates_plaintext = Parcellate(plaintext)

key = "This is a key!"
K = np.array(Binary_List(key), dtype=np.uint8).reshape(4, 4, order='F')
Keys = [K[0], K[1], K[2], K[3]]
Keys = Generate_All_Keys(Keys)

pt_blocks = [np.array(Binary_List(pt), dtype=np.uint8).reshape(4, 4, order='F') for pt in Parcellates_plaintext]

cipher_blocks = [Encryption(pt, Keys) for pt in pt_blocks]

cipher_string = ""
for block in cipher_blocks:
   cipher_string += "".join([chr(l) for l in block])
print(f"\nCiphertext: {cipher_string}")

Cipher = Parcellate(cipher_string)
ct_blocks = [np.array(Binary_List(ct), dtype=np.uint8).reshape(4, 4, order='F') for ct in Cipher]

Decrypted_blocks = [Decryption(ct, Keys) for ct in ct_blocks]

Decrypted_blocks_string = ""
for block in Decrypted_blocks:
   Decrypted_blocks_string += "".join([chr(l) for l in block])

print(f'Original text: {plaintext}')
print(f"Decrypted: {Decrypted_blocks_string}\n")


Ciphertext: .i.=U"äãkùÆ ràu½ôVöU¾a¾rÎÂ*ûÍkeø¶ÿeÅñÖ"I3g
Original text: HELLO THIS IS A TEST OF AES LONG MESSAGE!
Decrypted: HELLO THIS IS A TEST OF AES LONG MESSAGE!       

